In [27]:
import os
import json
import pandas as pd
import numpy as np


In [28]:
# Chemin complet vers le csv servant de source.
source_csv = '../data/metrics/raw_merge_metrics_dataset.csv'

# Chemin de stockage des csv
path = "../data/metrics/"

## <center>**Import du Dataset originel**<center>

In [29]:
# Création Dataframe
df = pd.read_csv(source_csv, index_col=0).reset_index(drop=True).dropna(axis=1, how="all")

C:\Users\Arthur\AppData\Local\Temp\ipykernel_14584\3036002175.py:2: DtypeWarning: Columns (2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(source_csv, index_col=0).reset_index(drop=True).dropna(axis=1, how="all")


In [30]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3510431 entries, 0 to 3510430
Data columns (total 17 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   id                         int64  
 1   source_events              object 
 2   message_events             object 
 3   timestamp_events           object 
 4   criticality_events         object 
 5   identification_events      object 
 6   name_modules               object 
 7   type_modules               object 
 8   generation_modules         object 
 9   name_counters_modules      object 
 10  value_counters_modules     int64  
 11  name_connected_operators   object 
 12  level_connected_operators  object 
 13  status                     object 
 14  created_at                 object 
 15  varnishLevelsTargetvolume  float64
 16  varnishLevelsTotalvolume   int64  
dtypes: float64(1), int64(3), object(13)
memory usage: 455.3+ MB


,id,source_events,message_events,timestamp_events,criticality_events,identification_events,name_modules,type_modules,generation_modules,name_counters_modules,value_counters_modules,name_connected_operators,level_connected_operators,status,created_at,varnishLevelsTargetvolume,varnishLevelsTotalvolume
0,4170152,iFoil,JV-Ti non prêt : impression impossible,2022-04-15T06:06:56.278Z,INFO,391,iFoil L,iFoil,Gen. 2,Total Pages Counter,22881,Viktor,Operator,IDLE,2022-04-15 06:06:35.404000,36192.322612,100000
1,4170152,iFoil,JV-Ti non prêt : impression impossible,2022-04-15T06:06:56.278Z,INFO,391,iFoil L,iFoil,Gen. 2,Foiled Pages Counter,31092,Viktor,Operator,IDLE,2022-04-15 06:06:35.404000,36192.322612,100000
2,4170152,PLC,En attente,2022-04-15T06:06:56.418Z,INFO,330,Print Engine 1,Varnish Printer,NaN,3D Varnish Counter,1792992,Viktor,Operator,IDLE,2022-04-15 06:06:35.404000,36192.322612,100000
3,4171311,iFoil,En attente,2022-04-15T06:40:40.321Z,INFO,330,iFoil L,iFoil,Gen. 2,Total Pages Counter,22883,Viktor,Operator,IDLE,2022-04-15 06:40:19.459000,36192.263131,100000
4,4171311,iFoil,En attente,2022-04-15T06:40:40.321Z,INFO,330,iFoil L,iFoil,Gen. 2,Foiled Pages Counter,31092,Viktor,Operator,IDLE,2022-04-15 06:40:19.459000,36192.263131,100000


## <center>**Préparation du Dataset**<center>

### I - Imputation des données manquantes et suppression des colonnes inutiles 

In [31]:
# création d'une liste de colonnes à suppr
drop_col = ["id", "message_events"]

In [32]:
# Renommage de la colonne "timestamp_events"
if "timestamp_events" in df :
    df = df.rename(columns={"timestamp_events" : "timestamp"})

In [33]:
# Remplacement les données manquants dans la colonne "timestamp_events" par les timestamp de la colonne "created_at"
df.timestamp = df.timestamp.fillna(df["created_at"])

In [34]:
# ajout des colonnes redondante à la liste de colonnes a suppr
if "created_at" not in drop_col :
    drop_col.append("created_at")
if "name_modules" not in drop_col :
    drop_col.append("name_modules")
if "generation_modules" not in drop_col :
    drop_col.append("generation_modules")
if "status" not in drop_col :
    drop_col.append("status")

print(drop_col)


['id', 'message_events', 'created_at', 'name_modules', 'generation_modules', 'status']


In [35]:
df = df.drop(drop_col, axis=1)

In [36]:
df.head()

,source_events,timestamp,criticality_events,identification_events,type_modules,name_counters_modules,value_counters_modules,name_connected_operators,level_connected_operators,varnishLevelsTargetvolume,varnishLevelsTotalvolume
0,iFoil,2022-04-15T06:06:56.278Z,INFO,391,iFoil,Total Pages Counter,22881,Viktor,Operator,36192.322612,100000
1,iFoil,2022-04-15T06:06:56.278Z,INFO,391,iFoil,Foiled Pages Counter,31092,Viktor,Operator,36192.322612,100000
2,PLC,2022-04-15T06:06:56.418Z,INFO,330,Varnish Printer,3D Varnish Counter,1792992,Viktor,Operator,36192.322612,100000
3,iFoil,2022-04-15T06:40:40.321Z,INFO,330,iFoil,Total Pages Counter,22883,Viktor,Operator,36192.263131,100000
4,iFoil,2022-04-15T06:40:40.321Z,INFO,330,iFoil,Foiled Pages Counter,31092,Viktor,Operator,36192.263131,100000


In [37]:
# Vérification de la présence de NaN dans la colonne "identification_events"
df["identification_events"].isnull().values.any()

True

In [38]:
df.identification_events = df.identification_events.replace(np.nan, 200)

### II - Modification des données textuelles en données numériques et encodage

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3510431 entries, 0 to 3510430
Data columns (total 11 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   source_events              object 
 1   timestamp                  object 
 2   criticality_events         object 
 3   identification_events      object 
 4   type_modules               object 
 5   name_counters_modules      object 
 6   value_counters_modules     int64  
 7   name_connected_operators   object 
 8   level_connected_operators  object 
 9   varnishLevelsTargetvolume  float64
 10  varnishLevelsTotalvolume   int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 294.6+ MB


#### 1 - Colonne "timestamp"

In [40]:
# transformation en datetime
df.timestamp = pd.to_datetime(df.timestamp, utc=True)

#### 2 - Colonne "criticality_events"

In [41]:
# on remplace dans le dataframe les valeurs du type 'str' avec un code
df.criticality_events = df.criticality_events.fillna("UNDEFINED")

#### 3 - Colonne "identification_events"

In [42]:
df.identification_events.unique()

array(['391', '330', '377', 200, '407', '332', '331', '313', '333', '376',
       '454', '323', '480', '386', '479', '372', 'Kernel_Error', '344',
       '343', '445', '352', '324', '321', '385', '354', '355', '440',
       '371', '358', '357', '334', '381', '311', '453',
       'iFoil communication error', '472', '356', '405', '387', '447',
       '0', 'RCB communication error', '380', '388', '460',
       'ICB communication error', '411', '408', '471', '325', '320',
       '430', '406', '417', '444', '329', '446',
       'Pilot communication error', '359', '315', '384', '322', '345',
       '351', '349', '418', '389', '476', '350', '475', '466', '416',
       '346', 445.0, 391.0, '327', 430.0, '2', '326', '419', 333.0, 332.0,
       330.0, 334.0, 331.0, 377.0, 315.0, 406.0, 407.0, 376.0, 325.0,
       454.0, 313.0, 352.0, 344.0, 385.0, 371.0, 386.0], dtype=object)

In [43]:
# Encodage des 5 id textuelles
events_id = []
str_code_dict = {}
str_code = 500
for id in list(df['identification_events'].unique()) :
    try:
        events_id.append(int(id))
    except ValueError:
        str_code_dict[id] = str_code
        events_id.append(str_code)
        str_code += 1
str_code_dict

{'Kernel_Error': 500,
 'iFoil communication error': 501,
 'RCB communication error': 502,
 'ICB communication error': 503,
 'Pilot communication error': 504}

In [44]:
# on remplace dans le dataframe les valeurs des id textuelles par leur code correspondant
df.identification_events = df.identification_events.replace(str_code_dict)
# on converti toutes les valeurs en int64
df.identification_events = pd.to_numeric(df.identification_events).astype('int64')

In [45]:
df.identification_events.unique()

array([391, 330, 377, 200, 407, 332, 331, 313, 333, 376, 454, 323, 480,
       386, 479, 372, 500, 344, 343, 445, 352, 324, 321, 385, 354, 355,
       440, 371, 358, 357, 334, 381, 311, 453, 501, 472, 356, 405, 387,
       447,   0, 502, 380, 388, 460, 503, 411, 408, 471, 325, 320, 430,
       406, 417, 444, 329, 446, 504, 359, 315, 384, 322, 345, 351, 349,
       418, 389, 476, 350, 475, 466, 416, 346, 327,   2, 326, 419],
      dtype=int64)

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3510431 entries, 0 to 3510430
Data columns (total 11 columns):
 #   Column                     Dtype              
---  ------                     -----              
 0   source_events              object             
 1   timestamp                  datetime64[ns, UTC]
 2   criticality_events         object             
 3   identification_events      int64              
 4   type_modules               object             
 5   name_counters_modules      object             
 6   value_counters_modules     int64              
 7   name_connected_operators   object             
 8   level_connected_operators  object             
 9   varnishLevelsTargetvolume  float64            
 10  varnishLevelsTotalvolume   int64              
dtypes: datetime64[ns, UTC](1), float64(1), int64(3), object(6)
memory usage: 294.6+ MB


In [47]:
df.head()

,source_events,timestamp,criticality_events,identification_events,type_modules,name_counters_modules,value_counters_modules,name_connected_operators,level_connected_operators,varnishLevelsTargetvolume,varnishLevelsTotalvolume
0,iFoil,2022-04-15 06:06:56.278000+00:00,INFO,391,iFoil,Total Pages Counter,22881,Viktor,Operator,36192.322612,100000
1,iFoil,2022-04-15 06:06:56.278000+00:00,INFO,391,iFoil,Foiled Pages Counter,31092,Viktor,Operator,36192.322612,100000
2,PLC,2022-04-15 06:06:56.418000+00:00,INFO,330,Varnish Printer,3D Varnish Counter,1792992,Viktor,Operator,36192.322612,100000
3,iFoil,2022-04-15 06:40:40.321000+00:00,INFO,330,iFoil,Total Pages Counter,22883,Viktor,Operator,36192.263131,100000
4,iFoil,2022-04-15 06:40:40.321000+00:00,INFO,330,iFoil,Foiled Pages Counter,31092,Viktor,Operator,36192.263131,100000


#### 4 - Colonne "varnishLevelsTargetvolume"

In [48]:
# on converti les float en entier 64
df.varnishLevelsTargetvolume = pd.to_numeric(df.varnishLevelsTargetvolume).astype('int64')

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3510431 entries, 0 to 3510430
Data columns (total 11 columns):
 #   Column                     Dtype              
---  ------                     -----              
 0   source_events              object             
 1   timestamp                  datetime64[ns, UTC]
 2   criticality_events         object             
 3   identification_events      int64              
 4   type_modules               object             
 5   name_counters_modules      object             
 6   value_counters_modules     int64              
 7   name_connected_operators   object             
 8   level_connected_operators  object             
 9   varnishLevelsTargetvolume  int64              
 10  varnishLevelsTotalvolume   int64              
dtypes: datetime64[ns, UTC](1), int64(4), object(6)
memory usage: 294.6+ MB


In [50]:
df.source_events.unique()

array(['iFoil', 'PLC', nan, 'Kernel', 'RCB n°1', 'RCB n°2', 'ICB n°1',
       'RCB n°3', 'ICB n°5', 'ICB n°7', 'ICB n°8', 'Pilot', 'ICB n°4',
       'ICB n°2', 'ICB n°6'], dtype=object)

In [51]:
df.type_modules.unique()

array(['iFoil', 'Varnish Printer'], dtype=object)

### III - Sauvegarde du nouveau dataframe pour réutilisation

In [52]:
df.to_csv("../data/metrics/Arthur_dataset_V1.csv")